# 元編程

In [ ]:
https://www.liaoxuefeng.com/wiki/0014316089557264a6b348958f449949df42a6d3a2e542c000/0014319106919344c4ef8b1e04c48778bb45796e0335839000

# type 動態生成類

我们说class的定义是运行时动态创建的，而创建class的方法就是使用type()函数。

type()函数既可以返回一个对象的类型，又可以创建出新的类型，

比如，我们可以通过type()函数创建出Hello类，

而无需通过class Hello(object)...的定义：

In [1]:
def fn(self, name='world'): # 先定义函数
    print('Hello, %s.' % name)
    
Hello = type('Hello', (object,), dict(hello=fn)) # 创建Hello class

In [2]:
Hello

__main__.Hello

要创建一个class对象，type()函数依次传入3个参数：

1. class的名称；

2. 继承的父类集合，注意Python支持多重继承，如果只有一个父类，别忘了tuple的单元素写法；

3. class的方法名称与函数绑定，这里我们把函数fn绑定到方法名hello上。

# metaclass 元類

当我们定义了类以后，就可以根据这个类创建出实例，

所以：先定义类，然后创建实例。

但是如果我们想创建出类呢？那就必须根据metaclass创建出类，所以：先定义metaclass，然后创建类。

连接起来就是：

    先定义metaclass，就可以创建类，最后创建实例。

    所以，metaclass允许你创建类或者修改类。

我们先看一个简单的例子，

这个metaclass可以给我们自定义的MyList增加一个add方法：

定义ListMetaclass，按照默认习惯，

metaclass的类名总是以Metaclass结尾，

以便清楚地表示这是一个metaclass：

In [3]:
# metaclass是类的模板，所以必须从`type`类型派生：
class ListMetaclass(type):
    def __new__(cls, name, bases, attrs):
        attrs['add'] = lambda self, value: self.append(value)
        return type.__new__(cls, name, bases, attrs)

In [ ]:
有了ListMetaclass，

我们在定义类的时候还要指示使用ListMetaclass来定制类，

传入关键字参数metaclass：

In [5]:
class MyList(list, metaclass=ListMetaclass):
    pass

当我们传入关键字参数metaclass时，魔术就生效了，

它指示Python解释器在创建MyList时，

要通过ListMetaclass.\_\_new\_\_()来创建，

在此，我们可以修改类的定义，

比如，加上新的方法，然后，返回修改后的定义。

\_\_new\_\_()方法接收到的参数依次是：

1. 当前准备创建的类的对象；

2. 类的名字；

3. 类继承的父类集合；

4. 类的方法集合。

测试一下MyList是否可以调用add()方法：

In [7]:
L = MyList()
L.add(1)
L

[1]

动态修改有什么意义？

直接在MyList定义中写上add()方法不是更简单吗？

正常情况下，确实应该直接写，通过metaclass修改纯属变态。

但是，总会遇到需要通过metaclass修改类定义的。

ORM就是一个典型的例子。

# ORM

ORM全称“Object Relational Mapping”，

即对象-关系映射，就是把关系数据库的一行映射为一个对象，

也就是一个类对应一个表，这样，写代码更简单，不用直接操作SQL语句。

要编写一个ORM框架，所有的类都只能动态定义，

因为只有使用者才能根据表的结构定义出对应的类来。

编写底层模块的第一步，就是先把调用接口写出来。

比如，使用者如果使用这个ORM框架，

想定义一个User类来操作对应的数据库表User

In [37]:
class User(Model):
    # 定义类的属性到列的映射：
    id = IntegerField('id')
    name = StringField('username')
    email = StringField('email')
    password = StringField('password')

# 创建一个实例：
u = User(id=12345, name='Michael', email='test@orm.org', password='my-pwd')
# 保存到数据库：
u.save()

Found model: User
Found mapping: id ==> <IntegerField:id>
Found mapping: name ==> <StringField:username>
Found mapping: email ==> <StringField:email>
Found mapping: password ==> <StringField:password>
SQL: insert into User (id,username,email,password) values (id,name,email,password)
ARGS: [12345, 'Michael', 'test@orm.org', 'my-pwd']


## Field

首先来定义Field类，它负责保存数据库表的字段名和字段类型：

In [10]:
class Field(object):

    def __init__(self, name, column_type):
        self.name = name
        self.column_type = column_type

    def __str__(self):
        return '<%s:%s>' % (self.__class__.__name__, self.name)

### StringField，IntegerField ...

在Field的基础上，进一步定义各种类型的Field，比如StringField，IntegerField等等：

In [11]:
class StringField(Field):

    def __init__(self, name):
        super(StringField, self).__init__(name, 'varchar(100)')

class IntegerField(Field):

    def __init__(self, name):
        super(IntegerField, self).__init__(name, 'bigint')

In [12]:
class ModelMetaclass(type):

    def __new__(cls, name, bases, attrs):
        if name=='Model':
            return type.__new__(cls, name, bases, attrs)
        print('Found model: %s' % name)
        mappings = dict()
        for k, v in attrs.items():
            if isinstance(v, Field):
                print('Found mapping: %s ==> %s' % (k, v))
                mappings[k] = v
        for k in mappings.keys():
            attrs.pop(k)
        attrs['__mappings__'] = mappings # 保存属性和列的映射关系
        attrs['__table__'] = name # 假设表名和类名一致
        return type.__new__(cls, name, bases, attrs)

In [42]:
class Model(dict, metaclass=ModelMetaclass):

    def __init__(self, **kw):
        super(Model, self).__init__(**kw)

    def __getattr__(self, key):
        try:
            return self[key]
        except KeyError:
            raise AttributeError(r"'Model' object has no attribute '%s'" % key)

    def __setattr__(self, key, value):
        self[key] = value

    def save(self):
        fields = []
        params = []
        args = []
        for k, v in self.__mappings__.items():
            fields.append(v.name)
            params.append(str(self[k]))
            args.append(getattr(self, k, None))
        sql = 'insert into %s (%s) values (%s)' % (self.__table__, ','.join(fields), ','.join(params))
        print('SQL: %s' % sql)
        print('ARGS: %s' % str(args))

当用户定义一个class User(Model)时，Python解释器首先在当前类User的定义中查找metaclass，如果没有找到，就继续在父类Model中查找metaclass，找到了，就使用Model中定义的metaclass的ModelMetaclass来创建User类，也就是说，metaclass可以隐式地继承到子类，但子类自己却感觉不到。

在ModelMetaclass中，一共做了几件事情：

1. 排除掉对Model类的修改；

2. 在当前类（比如User）中查找定义的类的所有属性，如果找到一个Field属性，就把它保存到一个\_\_mappings\_\_的dict中，同时从类属性中删除该Field属性，否则，容易造成运行时错误（实例的属性会遮盖类的同名属性）；

3. 把表名保存到\_\_table\_\_中，这里简化为表名默认为类名。

4. 在Model类中，就可以定义各种操作数据库的方法，比如save()，delete()，find()，update等等。

我们实现了save()方法，把一个实例保存到数据库中。因为有表名，属性到字段的映射和属性值的集合，就可以构造出INSERT语句。

编写代码试试：

In [43]:
class User(Model):
    # 定义类的属性到列的映射：
    id = IntegerField('id')
    name = StringField('username')
    email = StringField('email')
    password = StringField('password')

# 创建一个实例：
u = User(id=12345, name='Michael', email='test@orm.org', password='my-pwd')
# 保存到数据库：
u.save()

Found model: User
Found mapping: id ==> <IntegerField:id>
Found mapping: name ==> <StringField:username>
Found mapping: email ==> <StringField:email>
Found mapping: password ==> <StringField:password>
SQL: insert into User (id,username,email,password) values (12345,Michael,test@orm.org,my-pwd)
ARGS: [12345, 'Michael', 'test@orm.org', 'my-pwd']
